In [1]:
import qiskit
qiskit.__version__
from statistics import mode


In [2]:
import numpy as np
import os
import time
from pylab import *
from math import pi
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister,transpile


In [3]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()

In [4]:
x_data = iris["data"]
y_data = iris['target']

# Dataset

se usa en este programa el *iris-dataset* que tiene 3 clases ***"Iris-setosa","Iris-versicolor","Iris-virginica"***, cada uno tiene 50 instancias, dando un total de *150*. 

In [5]:
def indexPosicion(value):
    list_bin = []
    temp = bin(value)[2:]
    temp = temp[::-1]
    for i in range(len(temp)):
        if temp[i] == '1':
            list_bin.append(i)
    return list_bin

In [6]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mcx(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "$U_s$"
    return U_s

In [7]:
def qram(qc,address,ancilla,oracle,data,train_set,y_train,len_arr):
    if len_arr > len(train_set):
        len_arr = len(train_set)
    for i in range(len_arr):
        x_gates_array = indexPosicion(i)
        if x_gates_array:
            qc.x(address[x_gates_array])
            qc.mcx(address,ancilla)
            angles = train_set[i]
            for index in range(len(angles)):
                qc.crz(angles[index],ancilla, data[index])
            qc.mcx(address,ancilla)
            qc.x(address[x_gates_array])
        else:
            qc.mcx(address,ancilla)
            angles = train_set[i]
            for index in range(len(angles)):
                qc.crz(angles[index],ancilla, data[index])
            qc.mcx(address,ancilla)
 
        qc.barrier()

In [8]:
def qram_d(qc,address,ancilla,data,train_set,len_arr):
    if len_arr > len(train_set):
        len_arr = len(train_set)
    for i in range(len_arr):
        x_gates_array = indexPosicion(i)
        if x_gates_array:
            qc.x(address[x_gates_array])
            qc.mcx(address,ancilla)
            angles = train_set[i]
            for index in range(len(angles)):
                qc.crz(-angles[index],ancilla, data[index])
            qc.mcx(address,ancilla)
            qc.x(address[x_gates_array])
        else:
            qc.mcx(address,ancilla)
            angles = train_set[i]
            for index in range(len(angles)):
                qc.crz(-angles[index],ancilla, data[index])
            qc.mcx(address,ancilla)
        qc.barrier()

In [9]:
def knn_1_V2(input_a, train_set,y_train,size):
    address = QuantumRegister(size)
    ancilla = QuantumRegister(1)
    data_train = QuantumRegister(4)
    data_test = QuantumRegister(4)
    swap_test = QuantumRegister(1)
    oracle = QuantumRegister(1)
    c = ClassicalRegister(size)
    c_oracle = ClassicalRegister(1)
    qc = QuantumCircuit(address, ancilla, data_train, data_test, swap_test,oracle, c,c_oracle)

    qc.h(address)
    qc.x(oracle)
    qc.h(oracle)
    qc.h(data_train)
    qc.h(data_test)
    qc.h(swap_test)
    
    qc.barrier()
    
    
    n = (2**size)
    for index in range(1):
        qram(qc,address,ancilla,oracle,data_train,train_set,y_train,n)
        qc.barrier()
        angles = input_a

        for index in range(len(angles)):
            qc.rz(angles[index],data_test[index]) 
        qc.barrier()
        for i in range(4):
            qc.cswap(swap_test[0],data_train[i],data_test[i])
        qc.h(swap_test)
        qc.barrier()

        qc.measure(swap_test,c_oracle)
        qc.x(oracle).c_if(c_oracle[0], 0)
        qc.h(data_test)

        qc.barrier()
        qram_d(qc,address,ancilla,data_train,train_set,n)
        qc.barrier()    
        qc.append(diffuser(size),address) 
        qc.barrier() 
        qc.h(data_train)
        
    qc.x(address)
    qc.measure(address,c)
    
    return qc

In [10]:
import operator
from sklearn.metrics import confusion_matrix
from qiskit_aer import AerSimulator

acc_1 = []
acc_3 = []
acc_5 = []
acc_7 = []
acc_9 = []
acc_11 = []
train_size = 0.7
seed = [1,2,4,5,6,10,11,12,13,18]

for iteration in range(10):
    seed_simulator = seed[iteration]
    x_train, x_test, y_train, y_test =train_test_split(x_data, y_data, train_size=train_size, random_state=seed_simulator)
    y_1 = []
    y_3 = []
    y_5 = []
    y_7 = []
    y_9 = []
    y_11 = []
    for size in range(7,8):
        result_lista = []
        y_pred = []
        counts_l = []
        index_k = len(y_test)
        for i in range(index_k):
            print(i)
            qc = knn_1_V2(x_test[i], x_train,y_train,size)
            result = AerSimulator(device='GPU',seed_simulator=seed_simulator).run(transpile(qc,basis_gates=["cx","rz","x","sx"],optimization_level=3), shots=10000).result()
            counts = result.get_counts(qc)
            order = sorted(counts.items(), key=lambda item: item[1], reverse=True)
            k = [1,3,5,7,9,11]
            list_ = []
            index = 0
            nn = 0
            while nn < k[len(k)-1]: 
                #print(order,order[index])
                if order[index][0][0] == '1' and int(order[index][0][2:],2)< 105:
                    list_.append(y_train[int(order[index][0][2:],2)])
                    nn+=1
                index+=1
            for k_ in k:
                if k_==1:
                    y_1.append(mode(list_[:k_]))
                if k_==3:
                    y_3.append(mode(list_[:k_]))
                elif k_==5:
                    y_5.append(mode(list_[:k_]))
                elif k_==7:
                    y_7.append(mode(list_[:k_]))
                elif k_==9:
                    y_9.append(mode(list_[:k_]))
                elif k_==11:
                    y_11.append(mode(list_[:k_]))
        
        acc_knn = acc_knn = accuracy_score(y_test,y_1)*100
        acc_1.append(acc_knn)
        print("1nn-train set: ",size,"Accuracy: ",acc_knn)
        
        acc_knn = acc_knn = accuracy_score(y_test,y_3)*100
        acc_3.append(acc_knn)
        print("3nn-train set: ",size,"Accuracy: ",acc_knn)
        
        acc_knn = acc_knn = accuracy_score(y_test,y_5)*100
        acc_5.append(acc_knn)
        print("5nn-train set: ",size,"Accuracy: ",acc_knn)
        
        acc_knn = acc_knn = accuracy_score(y_test,y_7)*100
        acc_7.append(acc_knn)
        print("7nn-train set: ",size,"Accuracy: ",acc_knn)
        
        acc_knn = acc_knn = accuracy_score(y_test,y_9)*100
        acc_9.append(acc_knn)
        print("9nn-train set: ",size,"Accuracy: ",acc_knn)

        acc_knn = acc_knn = accuracy_score(y_test,y_11)*100
        acc_11.append(acc_knn)
        print("11nn-train set: ",size,"Accuracy: ",acc_knn)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
1nn-train set:  7 Accuracy:  95.55555555555556
3nn-train set:  7 Accuracy:  97.77777777777777
5nn-train set:  7 Accuracy:  97.77777777777777
7nn-train set:  7 Accuracy:  97.77777777777777
9nn-train set:  7 Accuracy:  97.77777777777777
11nn-train set:  7 Accuracy:  100.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
1nn-train set:  7 Accuracy:  93.33333333333333
3nn-train set:  7 Accuracy:  95.55555555555556
5nn-train set:  7 Accuracy:  97.77777777777777
7nn-train set:  7 Accuracy:  97.77777777777777
9nn-train set:  7 Accuracy:  95.55555555555556
11nn-train set:  7 Accuracy:  95.55555555555556
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
1nn-train set:  7 Accuracy:  93.33333333333333
3nn-train set:  7 Accura

In [11]:
def mae_acc(acc):
    mean =  np.mean(np.asarray(acc))
    n = len(acc)
    summ = 0
    for i in range(n):
        summ += abs(mean - acc[i])
    return mean,summ/n

mae_acc(acc_1),mae_acc(acc_3),mae_acc(acc_5),mae_acc(acc_7),mae_acc(acc_9),mae_acc(acc_11)

((92.88888888888889, 2.0444444444444443),
 (94.44444444444446, 2.444444444444443),
 (96.0, 2.22222222222222),
 (95.99999999999997, 1.7777777777777757),
 (95.77777777777777, 1.24444444444444),
 (96.44444444444446, 1.5111111111111115))